# 통계학 개념

### 평균값, 중앙값, 최빈값

In [1]:
# 최빈<중앙<평균일 경우 우도>0(오른쪽으로 꼬리가 긴 형태)
# mode ==> mderesult 타입을 반환해주는데 mode와 count로 구성되어 있음
import numpy as np
from scipy import stats

np.random.seed(0)
data=np.random.randint(0,100,10000)

mean=np.mean(data); print('평균값: ', mean.round(2))
median=np.median(data); print('중앙값: ', median.round(2))
mode=stats.mode(data); print('최빈값: {} ({})'.format(mode[0][0], mode[1][0]))

평균값:  49.17
중앙값:  49.0
최빈값: 3 (125)


### 평균, 분산, 표준편차, 사분위수

In [2]:
# np.percentile ==> np.percentile(데이터,n) 데이터에서 n% 값 추출(크기 순)
import numpy as np
from statistics import variance,stdev

np.random.seed(0)
points=np.random.randint(0,100,20)

var=variance(points); print('분산: ', var)
std = stdev(points); print('표준편차: ', np.round(std,2))
range=np.max(points)-np.min(points); print('범위: ', range)

print('사분위수: ')
for val in [0,25,50,75,100]:
    quantile=np.percentile(points,val)
    print('{}% => {}'.format(val,quantile))

q1,q3=np.percentile(points,[25,75])
print('IQR: ', q3-q1)

분산:  662
표준편차:  25.73
범위:  79
사분위수: 
0% => 9.0
25% => 42.75
50% => 64.5
75% => 84.0
100% => 88.0
IQR:  41.25


### 단일표본 t검정
- 문제: 한 빵집에서 생산되는 식빵의 무게가 최소 200g이라고 주장할 경우, 표본 20개를 추출하여 구한 평균의 무게가 196g이고, 표준편차는 5.3g이었다면, 유의수준 5%(0.05)로 위의 주장을 기각할 수 있을까?

In [3]:
# ppf ==> percent point function으로 alpha,n에 해당하는 임계값 반환(검정통계량과 비교) 
# sf ==> survival function으로 검정통계량,n에 해당하는 p값 반환(alpha와 비교)
import numpy as np
from scipy import stats

x_bar,mu,sigma,n=196,200,5.3,20

t_sample=(x_bar-mu)/(sigma/np.sqrt(float(n)))
print('검정통계량: ', np.round(t_sample,2))

alpha=0.05
t_alpha=stats.t.ppf(alpha,n-1)
print('t-table로부터의 임계값: ',np.round(t_alpha,3))

p_val=stats.t.sf(np.abs(t_sample),n-1)
print('t-table의 아래쪽 꼬리 p 값: ', np.round(p_val,4))

검정통계량:  -3.38
t-table로부터의 임계값:  -1.729
t-table의 아래쪽 꼬리 p 값:  0.0016


### 표준정규분포
- 문제: 시험 점수가 정규분포를 따른다고 가정할 경우, 평균 점수가 56점이고 표준편차가 13.6인 경우, 75점 이상을 받는 학생들은 몇 %일까?

In [4]:
# cdf ==> culmative dencity function으로 z값까지의 면적(-무한대~z)값을 계산
import numpy as np
from scipy import stats

x, mu, sigma = 75, 56, 13.6

z = (x - mu) / sigma
print('z-score: ', np.round(z, 2))

p_val = 1 - stats.norm.cdf(z)
print('학생이 {}점 이상 받을 확률: {}%'.format(x,np.round(p_val*100,2)))

z-score:  1.4
학생이 75점 이상 받을 확률: 8.12%


### 카이제곱 독립성 검정
- 문제: 흡연은 운동에 영향을 미칠까?

In [5]:
# choice ==> data에 n개 만큼 랜덤하게 구성
import numpy as np
import pandas as pd
from scipy import stats

np.random.seed(0)

smoke = ['안함','가끔','매일','심함']
exercise = ['안함','가끔','매일']

data = {'smoke': np.random.choice(smoke,size=500), 'exercise': np.random.choice(exercise,size=500)}

df = pd.DataFrame(data, columns=['smoke','exercise'])
df.head()

,smoke,exercise
0,안함,안함
1,심함,안함
2,가끔,매일
3,안함,가끔
4,심함,가끔


In [6]:
# crosstab ==> 분할표 만들기(빈도수)
xtab=pd.crosstab(df.smoke,df.exercise)
xtab

exercise,가끔,매일,안함
smoke,,,
가끔,38,39,45
매일,34,44,33
심함,45,44,51
안함,47,41,39


In [7]:
# chi2_contingency ==> 카이제곱 값, p값, 자유도(dof), 기대값 등을 반환해줌
contg=stats.chi2_contingency(observed=xtab)
p_val=np.round(contg[1],3)
print('P-값: ',p_val)

P-값:  0.668


### ANOVA 분산분석
- 문제: 10명의 환자를 대상으로 A,B,C 세 가지 수면제 약효(수면시간)를 각각 테스트할 경우, 유의수준 0.05에서 A,B,C 수면제의 평균 수면시간은 동일한가?

In [8]:
import numpy as np
import pandas as pd
from scipy import stats

np.random.seed(0)
data=(np.random.rand(30).round(2)*10).reshape(-1,3)
df = pd.DataFrame(data=data,columns=['A','B','C'])
df

,A,B,C
0,5.5,7.2,6.0
1,5.4,4.2,6.5
2,4.4,8.9,9.6
3,3.8,7.9,5.3
4,5.7,9.3,0.7
5,0.9,0.2,8.3
6,7.8,8.7,9.8
7,8.0,4.6,7.8
8,1.2,6.4,1.4
9,9.4,5.2,4.1


In [9]:
# 분산분석은 f분포를 따르는 f통계량 사용(1개의 범주변수 사용->own_way, 2개 이상의 범주형 변수 사용)
# f_oneway ==> 1번째 값은 f통계량, 2번쨰 값은 p값
one_way_anova=stats.f_oneway(df.A,df.B,df.C)
print('통계량: {}, p-value: {}'.format(np.round(one_way_anova[0],2),np.round(one_way_anova[1],3)))

통계량: 0.34, p-value: 0.713
